# Event generation
Auhtor: Piotr Podlaski

## Load ROOT and necessary python packages

In [1]:
import ROOT
import os, sys, subprocess, time
import py8settings as py8s
from py8settings import PDF
import itertools as it
import ctypes

%jsroot on

Welcome to JupyROOT 6.24/04


## Compile pythia example code necessary to display events in graphical form

In [2]:
!cd pythia-examples; make main300; cd -

make: 'main300' is up to date.
/scratch_hdd/akalinow/Zajecia/2022-2023/Lato/Modern_Particle_Physics_Experiments/Modern_Particle_Physics_Experiments/03_Event_generation


## Play with Pythia8 settings and view events

In [3]:
cmnd_file = "settings.cmnd"

#we start fresh
if os.path.exists(cmnd_file):
    os.remove(cmnd_file)
os.system("touch "+cmnd_file)

#beam settings:
py8s.beam_settings(cmnd_file)

You can now set the parameters for the incoming beams:


interactive(children=(Dropdown(description='beam A id  [Beams:idA]', index=9, layout=Layout(width='750px'), op…

In [4]:
py8s.basic_settings(cmnd_file)

You can now select the number of events, settings for the random seed, and processes:


interactive(children=(IntText(value=100, description='# of events  [Main:numberOfEvents]', layout=Layout(width…

In [5]:
py8s.onoff_settings(cmnd_file)

You can switch on/off different parts of the simulation:


interactive(children=(Checkbox(value=True, description='multi-parton interactions  [PartonLevel:MPI]', layout=…

In [6]:
# Write settings for phase-space cuts to file.
py8s.pscuts_settings(cmnd_file, 2)

You can now select the phase-space cuts for 2 final-state particles:


interactive(children=(FloatText(value=4.0, description='minimum invariant mass in GeV  [PhaseSpace:mHatMin]', …

In [7]:
# Write settings for phase-space cuts to file.
py8s.shower_settings(cmnd_file)

You can now set the main shower parameters:


interactive(children=(FloatText(value=0.1365, description='alphaS(m_Z) for final-state shower (0.06 - 0.25)  […

In [7]:
# Write additional settings if needed.
#py8s.more_settings(cmnd_file, ["23:onMode = off", "23:onIfAny = 13"])

In [3]:
# Start pythia+dire.
command = "./pythia-examples/main300 --visualize_event --input "+cmnd_file  #+ ">> /dev/null"
subprocess.call(["bash","-c",command]);

NameError: name 'cmnd_file' is not defined

In [24]:
subprocess.call(["bash","-c","dot -Tpdf event-"+cmnd_file+".dot -o event-"+cmnd_file+".pdf"])
PDF("event-"+cmnd_file+".pdf",size=(1280,1024))

# Run Pythia8 from ROOT interface

First we prepare and initialize basic pythia configuration for generation of $p+p$ events at 14 TeV:
 * define helper function to suppress pythia messages
 * create `TPythia8` object
 * use `HardQCD:all` physics processes
 * set random seed to 42 (dummy number)
 * initialize pythia to simulate $p+p$ collisions at 14 TeV

In [4]:
def makePythiaSilent(pythia):
    pythia.ReadString("Init:showMultipartonInteractions = off")
    pythia.ReadString("Init:showChangedParticleData = off")
    pythia.ReadString("Init:showProcesses = off")
    pythia.ReadString("Init:showChangedSettings = off")
    pythia.ReadString("Next:numberShowInfo = 0")
    pythia.ReadString("Next:numberShowProcess = 0")
    pythia.ReadString("Next:numberShowEvent = 0")
    pythia.ReadString("Next:numberCount = 0")

pythia = ROOT.TPythia8(False)
makePythiaSilent(pythia)
pythia.ReadString("HardQCD:all = on")
pythia.ReadString("Random:setSeed = on")
pythia.ReadString("Random:seed = 0") # to ro randomize seed
pythia.Initialize(2212 , 2212 , 14000)
particles = ROOT.TClonesArray("TParticle", 1000) #prepare space for generated particles

 PYTHIA Warning in PhaseSpace2to2tauyz::trialKin: maximum for cross section violated  


To see how does it work we generate a single event and take a look at particle listings:

In [5]:
pythia.GenerateEvent()
pythia.EventListing()


 --------  PYTHIA Event Listing  (complete event)  ---------------------------------------------------------------------------------
 
    no         id  name            status     mothers   daughters     colours      p_x        p_y        p_z         e          m 
     0         90  (system)           -11     0     0     0     0     0     0      0.000      0.000      0.000  14000.000  14000.000
     1       2212  (p+)               -12     0     0    27     0     0     0      0.000      0.000   7000.000   7000.000      0.938
     2       2212  (p+)               -12     0     0    28     0     0     0      0.000      0.000  -7000.000   7000.000      0.938
     3         21  (g)                -21     7     0     5     6   101   102      0.000      0.000    564.498    564.498      0.000
     4         21  (g)                -21     8     8     5     6   103   101      0.000      0.000     -0.013      0.013      0.000
     5         21  (g)                -23     3     4     9     9   

### Prepare helper functions for particle identification

* write function to tell if particle is *final* (i.e. simulation of its evolution by pythia is finished). Hint: *final* particles have positive status codes
* add a second argument which will allow to filter by the particle type as well, keeping the function behaviour when only one argument is provided
* write a function that computes ivariant mass of the pair of particles
* the two defined functions assume that (a pair of) `TParticle` object(s) is passed to them

In [6]:
def isFinal(p,name=None):
    if name==None:
        return p.GetStatusCode()>0
    else:
        return p.GetStatusCode()>0 and p.GetName()==name


def getMinv(pair):
    p4_1=ROOT.TLorentzVector()
    p4_2=ROOT.TLorentzVector()
    pair[0].Momentum(p4_1)
    pair[1].Momentum(p4_2)
    p4=p4_1+p4_2
    return p4.M()

## Properties of hadron production in $p+p$ interactions

### Generate 10000 $p+p$ events at 14 TeV and draw $\frac{\mathrm{d} N}{\mathrm{d}\eta}$ and  $\frac{1}{p_T}\frac{\mathrm{d} N}{\mathrm{d}p_T}$ distributions of $p$, $\pi^+$ and $K^+$

* create list of particles' names in consideration
* use dictionaries to provide labels and colors for plotting, particle names should be used as keys
* in similar fashion initialize dictionaries of histograms (`TH1D`)
* invoke `TH1D::Sumw2()` method on each histogram to enable calculation of Poissonian errors
* generate events and fill histograms
* format histograms:
  * set markers to `ROOT.kFullCircle` for all histograms
  * set colors according to dictionaries defined at the begening
  * find out what is the maximal value of $y$ axis on $\eta$ and $p_T$ plots (among all particles)
  * scale histograms by $\frac{1}{\Delta x \cdot N}$, where $\Delta x$ is the bin width and $N$ is the number of events

In [7]:
nEvents=10000
parts=["proton", "pi+", "K+"]
labels={"proton": "p", "pi+": "#pi^{+}", "K+": "K^{+}"}
colors={"proton": ROOT.kBlack, "pi+": ROOT.kBlue, "K+": ROOT.kRed}
heta={}
hpt={}

for p in parts:
    heta[p]=ROOT.TH1D("eta"+p,"",120,-12,12)
    heta[p].Sumw2()
    hpt[p]=ROOT.TH1D("pt"+p,"",50,0,10)
    hpt[p].Sumw2()

def fillHistos(p):
    name=p.GetName()
    if name in parts:
        heta[name].Fill(p.Eta())
        hpt[name].Fill(p.Pt(),1/(p.Pt()))

#generate events
for i in range(nEvents):
    pythia.GenerateEvent()
    pythia.ImportParticles(particles,"All")
    for p in particles:
        if isFinal(p):
            fillHistos(p)
#format and normalize histograms, find ranges on y axes
maxeta=0
maxpt=0
for p in parts:
    heta[p].SetMarkerStyle(ROOT.kFullCircle)
    heta[p].SetMarkerColor(colors[p])
    bw=heta[p].GetBinWidth(1)
    heta[p].Scale(1./(bw*nEvents))
    maxeta=max(maxeta,heta[p].GetMaximum())
    
    hpt[p].SetMarkerStyle(ROOT.kFullCircle)
    hpt[p].SetMarkerColor(colors[p])
    bw=hpt[p].GetBinWidth(1)
    hpt[p].Scale(1./(bw*nEvents))
    maxpt=max(maxpt,hpt[p].GetMaximum())

 PYTHIA Warning in Pythia::check: energy-momentum not quite conserved  
 PYTHIA Warning in Pythia::check: not quite matched particle energy/momentum/mass  
 PYTHIA Warning in SimpleSpaceShower::pT2nextQCD: weight above unity  
 PYTHIA Warning in StringFragmentation::fragmentToJunction: bad convergence junction rest frame  
 PYTHIA Error in StringFragmentation::fragment: stuck in joining  
 PYTHIA Error in Pythia::next: hadronLevel failed; try again  


Warning in <TParticle::SetPdgCode>: PDG code 9942033 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 9942033 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 9942033 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 9942033 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 9942033 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 9942033 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG


Now, once simuation is complete and histograms are filled we proceed to drawing them:
* create `TCanvas` and divide it into two pads along $x$ axis
* prepare `TLegend` (the same for both plots for simplicity, sybbols for differend kinds of particles are identical)
* prepare *frames* for plotting (axes with correct labels and ranges)
  * for $\frac{1}{p_T}\frac{\mathrm{d} N}{\mathrm{d}p_T}$ set logarithmic scale on $y$ axis
* plot the histograms and and legends

In [8]:
c_etapt = ROOT.TCanvas("c_etapt","",1600,600)
c_etapt.Divide(2,1)

In [9]:
leg=ROOT.TLegend(0.8,0.75,0.9,0.9)
for p in parts:
    leg.AddEntry(hpt[p], labels[p])

c_etapt.cd(1).SetLogy()
frame_pt=c_etapt.cd(1).DrawFrame(0,maxpt*1e-8,10,10*maxpt)
frame_pt.GetXaxis().SetTitle("p_{T} [GeV/c]")
frame_pt.GetXaxis().SetTitleOffset(1.5)
frame_pt.GetYaxis().SetTitleSize(15)
frame_pt.GetYaxis().SetTitle("#frac{1}{p_{T}}#frac{dN}{dp_{T}} [GeV/c]^{-2}")
frame_pt.Draw()
for p in parts:
    hpt[p].Draw("p same")
leg.Draw()

c_etapt.cd(2)
frame_eta=c_etapt.cd(2).DrawFrame(-12,0,12,1.1*maxeta)
frame_eta.GetXaxis().SetTitle("#eta")
frame_eta.GetYaxis().SetTitleSize(15)
frame_eta.GetYaxis().SetTitle("#frac{dN}{d#eta} ")
frame_eta.Draw()
for p in parts:
    heta[p].Draw("p same")
    
leg.Draw()

c_etapt.Draw()

## $K^*$ meson production

### Generate 10000 $p+p$ events at 14 TeV and look for $K^*$ meson by plotting invariant mass of $\pi^-K^+$ pairs. Compare this spectrum with invariant mass spectrum of produced $K^*$'s decaying into kaon and pion.

* create two one dimensional histograms to hold invariant masses of  $\pi^--K^+$ pairs and $K^*$ mesons (300 bins each, range (0.5,2) GeV/$c^2$)
* during event generation in each event separately create lists of $\pi^-$'s and $K^+$'s and fill invariant mass histogram with invariant masses of all possible $\pi^--K^+$ pairs, use `getMinv` function defined above
  * HINT: use `itertools.product` (`it.product`), if `A` and `B` are lists, then `list(it.product(A,B))` is the list of all possible pairs where one element is from `A` and the other is from `B`
* second histogram should be filled with invariant masses of $K^*$'s decaying to pion and kaon, use `TParticle::GetDaughter(int n)` method (`n=0,1`, e.g. `p.GetDaugther(0), p.GetDaugther(1)`) to select the desired decay channel, this method returns *index* of the daughter particle - in order to access corresponding `TParticle` opject we have to directly access `particles[p.GetDaughter(0)]`
* create new `TCanvas`, present the two distributions on one plot with apropriate axes labels, different colors and legend

In [10]:
nEvents=10000
h=ROOT.TH1D("h_all","",300,0.5,2)
hpure=ROOT.TH1D("h_pure","",300,0.5,2)

def getKstarCandidates(parts):
    pim = [p for p in parts if isFinal(p,"pi-")]
    kp = [p for p in parts if isFinal(p,"K+")]
    return list(it.product(pim,kp))


for i in range(nEvents):
    pythia.GenerateEvent()
    pythia.ImportParticles(particles,"All")
    #for all possible pi- K+ pairs we fill invariant mass histogram
    for pair in getKstarCandidates(particles):
        h.Fill(getMinv(pair))
    #now we look for actual K*'s that decayed:
    for p in particles:
        if p.GetName()=="K*0":
            #we accept K* only if it decayed to pion and kaon
            d0=particles[p.GetDaughter(0)].GetName()
            d1=particles[p.GetDaughter(1)].GetName()
            if {d0,d1} != {"pi-","K+"}:
                continue
            p4=ROOT.TLorentzVector()
            p.Momentum(p4)
            hpure.Fill(p4.M())


c_kstar = ROOT.TCanvas("c","",800,600)
ROOT.gStyle.SetOptStat(0)
h.GetXaxis().SetTitle("M_{#pi^{-}K^{+}} [GeV/#it{c}^{2}]")
h.GetXaxis().SetTitleOffset(1.1)
h.GetYaxis().SetTitle("Entries")
h.SetLineWidth(2)
h.Draw()
hpure.SetLineWidth(2)
hpure.SetLineColor(ROOT.kRed)
hpure.Draw("same")
leg=ROOT.TLegend(0.5,0.7,0.9,0.9)
leg.AddEntry(h,"all #pi^{-}K^{+} pairs")
leg.AddEntry(hpure,"#pi^{-}K^{+} pairs from K^{*} decay")
leg.Draw()
c_kstar.Draw()

Warning in <TParticle::SetPdgCode>: PDG code 9941103 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 9941103 unknown from TDatabasePDG


## $Z$ boson production in $p+\bar{p}$ collisions

### Generate 10000 $p+\bar{p}$ events at 630 GeV of $Z$ production ($Z$ discovery at CERN SPS).

Setup:
* in order to look at $Z$ boson production we limit physics list of Pythia8 to include only hard $f\bar{f}\to Z \to f\bar{f}$ processes
* moreover, we will disable all $Z$ decay modes other than $Z\to e^+e^-$, in which the particle was discovered
* to speed up the simulation we disable hadronisation (we are interested only in leptonic final states)
* we also limit phase space to $\pm 20$ GeV around the $Z$ rest mass

To do:
* select candidates for products of $Z$ decay in each event and plot their invariant mass distribution in range 70-110 GeV
* check influence of final state radiation on the spectrum - generate additional 10000 events with FSR turned off
* normalize histograms so that their maxima are at $1$
* draw both distributions (as a histograms, option `"hist"`) on the same plot with axis labels and a legend
* draw $\frac{1}{p_T}\frac{dN}{dp_T}$ and $\frac{dN}{d\eta}$ spectra of produced $Z$ bosons (only for one dataset, with FSR on), with axis labels and proper normalisation, pick the $x$ axis ranges so that the whole distribution fits inside the plot
  * **hint:** $Z$ boson will occur multiple times in the event listing (why?), it will never be a final particle, pick only $Z$'s that appear *last* in each event
* what is the *mean* number $\langle N_Z\rangle$ of $Z$ bosons produced in an event, with what error? is this result surprising?
  * **hint:** $\langle N \rangle = \int\limits_0^\infty \frac{dN}{dp_T}\cdot dp_T = \int\limits_{-\infty}^\infty \frac{dN}{d\eta}\cdot d\eta$

In [11]:
nEvents=10000
pythia = ROOT.TPythia8(False) #False - disables banner
makePythiaSilent(pythia)
pythia.ReadString("WeakSingleBoson:ffbar2gmZ = on")
pythia.ReadString("23:onMode = off")
pythia.ReadString("23:onIfAny = 11")
pythia.ReadString("HadronLevel:all = off")
pythia.ReadString("PhaseSpace:mHatMin = 70.")
pythia.ReadString("PhaseSpace:mHatMax = 110.")
pythia.ReadString("Random:setSeed = on")
pythia.ReadString("Random:seed = 0")
pythia.Initialize(2212 , -2212 , 630);
particles = ROOT.TClonesArray("TParticle", 1000) #prepare space for generated particles

def getZCandidates(parts):
    ep = [p for p in parts if isFinal(p,"e+")]
    em = [p for p in parts if isFinal(p,"e-")]
    return list(it.product(ep,em))

h_rad=ROOT.TH1D("h_rad","",200,70,110)
h_norad=ROOT.TH1D("h_norad","",200,70,110)
h_pt=ROOT.TH1D("h_ptz","",100,0,50)
h_eta=ROOT.TH1D("h_etaz","",100,-10,10)


for i in range(nEvents):
    pythia.GenerateEvent()
    pythia.ImportParticles(particles,"All")
    for pair in getZCandidates(particles):
        h_rad.Fill(getMinv(pair))
    pt=0
    eta=0
    for p in particles:
        if p.GetName()=="Z0":
            pt=p.Pt()
            eta=p.Eta()
    h_pt.Fill(pt)
    h_eta.Fill(eta)

pythia = ROOT.TPythia8(False)
makePythiaSilent(pythia)
pythia.ReadString("WeakSingleBoson:ffbar2gmZ = on")
pythia.ReadString("23:onMode = off")
pythia.ReadString("23:onIfAny = 11")
pythia.ReadString("HadronLevel:all = off")
pythia.ReadString("PhaseSpace:mHatMin = 70.")
pythia.ReadString("PhaseSpace:mHatMax = 110.")
pythia.ReadString("PartonLevel:FSR = off")
pythia.ReadString("Random:setSeed = on")
pythia.ReadString("Random:seed = 0")
pythia.Initialize(2212 , -2212 , 630);

for i in range(nEvents):
    pythia.GenerateEvent()
    pythia.ImportParticles(particles,"All")
    for pair in getZCandidates(particles):
        h_norad.Fill(getMinv(pair))

c_z = ROOT.TCanvas("cz","",1600,1200)
c_z.Divide(2,2)
c_z.cd(1)

h_rad.Scale(1./(h_rad.GetMaximum()))
h_norad.Scale(1./(h_norad.GetMaximum()))

h_norad.SetLineColor(ROOT.kRed)
h_norad.SetLineWidth(2)
h_rad.SetLineWidth(2)
h_norad.GetXaxis().SetTitle("M_{e^{+}e^{-}}")
h_norad.GetXaxis().SetTitleOffset(1.4)
h_norad.GetYaxis().SetTitle("entries")
h_norad.Draw("hist")
h_rad.Draw("same hist")
leg=ROOT.TLegend(0.7,0.8,0.9,0.9)
leg.AddEntry(h_norad,"FSR OFF")
leg.AddEntry(h_rad,"FSR ON")
leg.Draw()

h_pt.SetMarkerStyle(ROOT.kFullCircle)
h_pt.SetMarkerColor(ROOT.kBlue)
h_pt.GetXaxis().SetTitle("p_{T} [GeV/c]")
h_pt.GetYaxis().SetTitle("#frac{dN}{dp_{T}} [GeV/c]^{-1}")
h_pt.GetXaxis().SetTitleOffset(1.5)
h_pt.GetYaxis().SetTitleSize(15)
bw=h_pt.GetBinWidth(1)
h_pt.Scale(1./(bw*nEvents))

h_eta.SetMarkerStyle(ROOT.kFullCircle)
h_eta.SetMarkerColor(ROOT.kBlue)
h_eta.GetXaxis().SetTitle("#eta")
h_eta.GetYaxis().SetTitle("#frac{dN}{d#eta}")
h_eta.GetYaxis().SetTitleSize(15)
bw=h_eta.GetBinWidth(1)
h_eta.Scale(1./(bw*nEvents))

c_z.cd(3)
h_eta.Draw()
c_z.cd(4)
h_pt.Draw()
c_z.Draw()


dnz=ctypes.c_double()
nz=h_pt.IntegralAndError(0,-1,dnz,"width")

print("Mean number of Z bosons per event: <N_Z>={:.3f} +/- {:.3f}".format(nz,dnz.value))

Mean number of Z bosons per event: <N_Z>=1.000 +/- 0.010
 *** TDatabasePDG::AddParticle: particle with PDGcode=90 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900110 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900210 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900220 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900330 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900440 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9902110 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9902210 already defined
 PYTHIA Warning in MultipartonInteractions::init: maximum increased by factor 1.019
 PYTHIA Warning in JunctionSplitting::SplitJunPairs: parallel junction state not allowed.  
 PYTHIA Warning in JunctionSplitting::CheckColours: Not possible to split junctions; making new colours  
 PYTHIA Error in BeamRemnants::add: failed

In [18]:
nEvents=10000
pythia = ROOT.TPythia8(False)                       # false to suppress banner
makePythiaSilent(pythia)                            # suppress most of pythia console output
pythia.ReadString("WeakSingleBoson:ffbar2gmZ = on") # select desired hard process
pythia.ReadString("23:onMode = off")                # disable all Z decay channels
pythia.ReadString("23:onIfAny = 11")                # enable Z decays to electrons
pythia.ReadString("HadronLevel:all = off")          # disable hadronisation
pythia.ReadString("PhaseSpace:mHatMin = 70.")       # lower phase space limit
pythia.ReadString("PhaseSpace:mHatMax = 110.")      # upper phase space limit
pythia.ReadString("PartonLevel:FSR = on")           # turn Final State Radiation on (note: it is on by default)
pythia.ReadString("Random:setSeed = on")            # random seed setup 
pythia.ReadString("Random:seed = 0")                # random seed setup
pythia.Initialize(2212 , -2212 , 630)               # initialize pythia
particles = ROOT.TClonesArray("TParticle", 1000)    # prepare space for generated particles

#your solution goes here

 *** TDatabasePDG::AddParticle: particle with PDGcode=90 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900110 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900210 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900220 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900330 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900440 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9902110 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9902210 already defined
 PYTHIA Warning in MultipartonInteractions::init: maximum increased by factor 1.046
